In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Activation,Dense,Dropout,Conv2D,Flatten,MaxPooling2D
from keras.datasets import cifar10
from keras import optimizers
from matplotlib import pyplot as plt


In [ ]:
from keras.callbacks import ReduceLROnPlateau

lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.5, min_lr=1e-5)


In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
from keras.datasets import fashion_mnist
from keras.utils import to_categorical
import numpy as np

# Load Fashion MNIST data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize and reshape
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
x_train = x_train.reshape(-1, 28, 28, 1)


In [ ]:
from keras.utils import to_categorical

y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)


In [ ]:
history = model.fit(x_train, y_train_cat,
                    validation_data=(x_test, y_test_cat),
                    epochs=10, batch_size=64, callbacks=[lr_reduction])



Epoch 1/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - accuracy: 0.7569 - loss: 0.6988 - val_accuracy: 0.8757 - val_loss: 0.3397 - learning_rate: 0.0010
Epoch 2/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8753 - loss: 0.3591 - val_accuracy: 0.8878 - val_loss: 0.3040 - learning_rate: 0.0010
Epoch 3/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8923 - loss: 0.3039 - val_accuracy: 0.8933 - val_loss: 0.2817 - learning_rate: 0.0010
Epoch 4/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.8995 - loss: 0.2807 - val_accuracy: 0.9019 - val_loss: 0.2688 - learning_rate: 0.0010
Epoch 5/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9052 - loss: 0.2580 - val_accuracy: 0.9030 - val_loss: 0.2624 - learning_rate: 0.0010
Epoch 6/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9090 - loss: 0.2463 - val_accuracy: 0.8998 - val_loss: 0.2681 - learning_rate: 0.0010
Epoch 7/10
938/938 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9127 - loss: 0.2374 - 

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Model creation function
def create_model(optimizer='adam'):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model


In [ ]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.utils import to_categorical
import numpy as np


y_train_cat = to_categorical(y_train, 10)


param_grid = {
    'batch_size': [32, 64],
    'epochs': [5],
    'optimizer': ['adam', 'rmsprop']
}

# Define a function to fit the model using GridSearchCV
def fit_model(optimizer='adam', batch_size=32, epochs=5):
    model = create_model(optimizer=optimizer)
    history = model.fit(x_train, y_train_cat, batch_size=batch_size, epochs=epochs, verbose=0)
    return history.history['accuracy'][-1]  # Return the final accuracy as the score for GridSearchCV

# Manually perform GridSearchCV
best_score = 0
best_params = {}

for optimizer in param_grid['optimizer']:
    for batch_size in param_grid['batch_size']:
        for epochs in param_grid['epochs']:
            score = fit_model(optimizer=optimizer, batch_size=batch_size, epochs=epochs)
            print(f"Score for optimizer={optimizer}, batch_size={batch_size}, epochs={epochs}: {score}")
            if score > best_score:
                best_score = score
                best_params = {'optimizer': optimizer, 'batch_size': batch_size, 'epochs': epochs}

print("Best parameters found: ", best_params)


Score for optimizer=adam, batch_size=32, epochs=5: 0.8995500206947327
Score for optimizer=adam, batch_size=64, epochs=5: 0.9023833274841309
Score for optimizer=rmsprop, batch_size=32, epochs=5: 0.8951833248138428
Score for optimizer=rmsprop, batch_size=64, epochs=5: 0.9003333449363708
Best parameters found:  {'optimizer': 'adam', 'batch_size': 64, 'epochs': 5}


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np

y_pred = model.predict(x_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test_cat, axis=1)

cm = confusion_matrix(y_true, y_pred_classes)
ConfusionMatrixDisplay(cm).plot()
plt.title("Confusion Matrix")
plt.show()


In [ ]:
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Loss over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Accuracy over Epochs')
plt.legend()

plt.show()


In [ ]:
ReduceLROnPlateau reduces the learning rate when validation loss stops improving, preventing stagnation.
GridSearchCV finds the best optimizer  batch size, helping improve model generalization.
- These steps reduce overfitting  improve convergence  accuracy.


In [ ]:
from keras.datasets import fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

x_train = x_train.reshape(-1, 28, 28, 1) / 255.0
x_test = x_test.reshape(-1, 28, 28, 1) / 255.0

from keras.utils import to_categorical
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)


In [ ]:
class_correct = np.zeros(10)
class_total = np.zeros(10)
for i in range(len(y_test)):
    label = y_test[i]
    pred = y_pred_classes[i]
    if label == pred:
        class_correct[label] += 1
    class_total[label] += 1

accuracy_per_class = class_correct / class_total
plt.bar(range(10), accuracy_per_class)
plt.title("Class-wise Accuracy")
plt.xlabel("Class")
plt.ylabel("Accuracy")
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15, 3))
for i, ax in enumerate(axes):
    ax.imshow(x_test[i].reshape(28, 28), cmap='gray')
    ax.set_title(f"True: {y_test[i]}, Pred: {y_pred_classes[i]}")
    ax.axis('off')
plt.suptitle("Sample Predictions")
plt.show()
